# Lending disparities using Logistic Regression

**The story:** https://www.revealnews.org/article/for-people-of-color-banks-are-shutting-the-door-to-homeownership/

**Author:** Aaron Glantz and Emmanuel Martinez

**Topics:** Logistic regression, odds ratios

**Datasets**

* **philadelphia-mortgages.csv:** Philadelphia mortgage data for 2015
    - A subset of HMDA LAR data from [FFEIC](https://www.ffiec.gov/hmda/hmdaproducts.htm)
    - Codebook is `2015HMDACodeSheet.pdf`
    - A [guide to HMDA reporting](https://www.ffiec.gov/hmda/guide.htm)
    - I've massaged it slightly to make processing a bit easier
* **nhgis0006_ds233_20175_2017_tract.csv:**
    - Table B03002: Hispanic or Latino Origin by Race
    - 2013-2017 American Community Survey data US Census Bureau, from [NHGIS](https://data2.nhgis.org/main)
    - Codebook is `nhgis0006_ds233_20175_2017_tract_codebook.txt`
* **lending_disparities_whitepaper_180214.pdf:** the whitepaper outlining Reveal's methodology

## What's the goal?

Do banks provide mortgages at disparate rates between white applicants and people of color? We're going to look at the following variables to find out:

* Race/Ethnicity
    - Native American
    - Asian
    - Black
    - Native Hawaiian
    - Hispanic/Latino
    - Race and ethnicity were not reported
* Sex
* Whether there was a co-applicant
* Applicant’s annual income (includes co-applicant income)
* Loan amount
* Ratio between the loan amount and the applicant’s income
* Ratio between the median income of the census tract and the median income of the metro area
* Racial and ethnic breakdown by percentage for each census tract
* Regulating agency of the lending institution

# Setup

Import pandas as usual, but also import numpy. We'll need it for logarithms and exponents.

Some of our datasets have a lot of columns, so you'll also want to use `pd.set_option` to display up to 100 columns or so.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression,LogisticRegression

%matplotlib inline

In [2]:
pd.set_option("display.max_columns", 100)

# What is each row of our data?

If you aren't sure, you might need to look at either the whitepaper or the codebook. You'll need to look at them both eventually, so might as well get started now.

In [3]:
df=pd.read_csv('data/philadelphia-mortgages.csv',encoding='latin-1',dtype={'census_tract':'str'})
df.head()

,census_tract,county_code,state_code,applicant_sex,income,loan_amount,loan_type,property_type,occupancy,action_type,loan_purpose,agency_code,tract_to_msa_income_percent,applicant_race,co_applicant_sex
0,0101.00,101,42,3,26,5,1,1,1,4,2,5,97.09,6,5
1,0264.00,101,42,2,26,40,1,1,1,4,2,5,98.27,3,5
2,0281.00,101,42,2,22,20,1,1,1,5,2,5,72.28,6,5
3,0158.00,101,42,2,57,36,1,1,1,5,3,5,105.87,6,5
4,0358.00,101,42,1,80,34,1,1,1,1,3,5,139.62,5,2


In [4]:
df.shape

(60705, 15)

In [5]:
df.dtypes

census_tract                    object
county_code                      int64
state_code                       int64
applicant_sex                    int64
income                           int64
loan_amount                      int64
loan_type                        int64
property_type                    int64
occupancy                        int64
action_type                      int64
loan_purpose                     int64
agency_code                      int64
tract_to_msa_income_percent    float64
applicant_race                   int64
co_applicant_sex                 int64
dtype: object

In [6]:
df.columns

Index(['census_tract', 'county_code', 'state_code', 'applicant_sex', 'income',
       'loan_amount', 'loan_type', 'property_type', 'occupancy', 'action_type',
       'loan_purpose', 'agency_code', 'tract_to_msa_income_percent',
       'applicant_race', 'co_applicant_sex'],
      dtype='object')

In [6]:
df1 = pd.read_csv("data/nhgis0007_ds215_20155_2015_tract.csv", encoding='latin-1', dtype='str')
df1.head()

,GISJOIN,YEAR,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,PLACEA,TRACTA,BLKGRPA,CONCITA,AIANHHA,RES_ONLYA,TRUSTA,AITSCEA,ANRCA,CBSAA,CSAA,METDIVA,NECTAA,CNECTAA,NECTADIVA,UAA,CDCURRA,SLDUA,SLDLA,ZCTA5A,SUBMCDA,SDELMA,SDSECA,SDUNIA,PUMA5A,BTTRA,BTBGA,NAME_E,ADK5E001,ADK5E002,ADK5E003,ADK5E004,ADK5E005,ADK5E006,ADK5E007,ADK5E008,ADK5E009,ADK5E010,ADK5E011,ADK5E012,ADK5E013,ADK5E014,ADK5E015,ADK5E016,ADK5E017,ADK5E018,ADK5E019,ADK5E020,ADK5E021,NAME_M,ADK5M001,ADK5M002,ADK5M003,ADK5M004,ADK5M005,ADK5M006,ADK5M007,ADK5M008,ADK5M009,ADK5M010,ADK5M011,ADK5M012,ADK5M013,ADK5M014,ADK5M015,ADK5M016,ADK5M017,ADK5M018,ADK5M019,ADK5M020,ADK5M021
0,G0100010020100,2011-2015,NaN,NaN,Alabama,01,Autauga County,001,NaN,NaN,020100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Census Tract 201, Autauga County, Alabama",1948,1931,1703,150,6,12,0,0,60,0,60,17,17,0,0,0,0,0,0,0,0,"Census Tract 201, Autauga County, Alabama",203,212,229,126,8,16,11,11,44,11,44,21,21,11,11,11,11,11,11,11,11
1,G0100010020200,2011-2015,NaN,NaN,Alabama,01,Autauga County,001,NaN,NaN,020200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Census Tract 202, Autauga County, Alabama",2156,2139,872,1149,0,50,0,0,68,0,68,17,14,0,0,0,0,3,0,0,0,"Census Tract 202, Autauga County, Alabama",268,268,207,250,11,61,11,11,62,11,62,25,23,11,11,11,11,7,11,11,11
2,G0100010020300,2011-2015,NaN,NaN,Alabama,01,Autauga County,001,NaN,NaN,020300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Census Tract 203, Autauga County, Alabama",2968,2968,2212,551,15,41,8,0,141,0,141,0,0,0,0,0,0,0,0,0,0,"Census Tract 203, Autauga County, Alabama",404,404,372,190,22,62,14,11,135,11,135,11,11,11,11,11,11,11,11,11,11
3,G0100010020400,2011-2015,NaN,NaN,Alabama,01,Autauga County,001,NaN,NaN,020400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Census Tract 204, Autauga County, Alabama",4423,3959,3662,162,69,0,0,48,18,5,13,464,30,42,0,0,0,372,20,20,0,"Census Tract 204, Autauga County, Alabama",493,353,343,133,81,11,11,82,17,8,16,437,33,50,11,11,11,456,29,29,11
4,G0100010020500,2011-2015,NaN,NaN,Alabama,01,Autauga County,001,NaN,NaN,020500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Census Tract 205, Autauga County, Alabama",10763,10683,7368,2674,0,412,0,0,229,49,180,80,80,0,0,0,0,0,0,0,0,"Census Tract 205, Autauga County, Alabama",624,616,796,742,18,242,18,18,166,74,149,71,71,18,18,18,18,18,18,18,18


In [8]:
df1.shape

(74001, 80)

In [9]:
df1.dtypes

GISJOIN      object
YEAR         object
REGIONA      object
DIVISIONA    object
STATE        object
STATEA       object
COUNTY       object
COUNTYA      object
COUSUBA      object
PLACEA       object
TRACTA       object
BLKGRPA      object
CONCITA      object
AIANHHA      object
RES_ONLYA    object
TRUSTA       object
AITSCEA      object
ANRCA        object
CBSAA        object
CSAA         object
METDIVA      object
NECTAA       object
CNECTAA      object
NECTADIVA    object
UAA          object
CDCURRA      object
SLDUA        object
SLDLA        object
ZCTA5A       object
SUBMCDA      object
              ...  
ADK5E014     object
ADK5E015     object
ADK5E016     object
ADK5E017     object
ADK5E018     object
ADK5E019     object
ADK5E020     object
ADK5E021     object
NAME_M       object
ADK5M001     object
ADK5M002     object
ADK5M003     object
ADK5M004     object
ADK5M005     object
ADK5M006     object
ADK5M007     object
ADK5M008     object
ADK5M009     object
ADK5M010     object


In [10]:
df1.columns

Index(['GISJOIN', 'YEAR', 'REGIONA', 'DIVISIONA', 'STATE', 'STATEA', 'COUNTY',
       'COUNTYA', 'COUSUBA', 'PLACEA', 'TRACTA', 'BLKGRPA', 'CONCITA',
       'AIANHHA', 'RES_ONLYA', 'TRUSTA', 'AITSCEA', 'ANRCA', 'CBSAA', 'CSAA',
       'METDIVA', 'NECTAA', 'CNECTAA', 'NECTADIVA', 'UAA', 'CDCURRA', 'SLDUA',
       'SLDLA', 'ZCTA5A', 'SUBMCDA', 'SDELMA', 'SDSECA', 'SDUNIA', 'PUMA5A',
       'BTTRA', 'BTBGA', 'NAME_E', 'ADK5E001', 'ADK5E002', 'ADK5E003',
       'ADK5E004', 'ADK5E005', 'ADK5E006', 'ADK5E007', 'ADK5E008', 'ADK5E009',
       'ADK5E010', 'ADK5E011', 'ADK5E012', 'ADK5E013', 'ADK5E014', 'ADK5E015',
       'ADK5E016', 'ADK5E017', 'ADK5E018', 'ADK5E019', 'ADK5E020', 'ADK5E021',
       'NAME_M', 'ADK5M001', 'ADK5M002', 'ADK5M003', 'ADK5M004', 'ADK5M005',
       'ADK5M006', 'ADK5M007', 'ADK5M008', 'ADK5M009', 'ADK5M010', 'ADK5M011',
       'ADK5M012', 'ADK5M013', 'ADK5M014', 'ADK5M015', 'ADK5M016', 'ADK5M017',
       'ADK5M018', 'ADK5M019', 'ADK5M020', 'ADK5M021'],
      dtype='obje

# Read in your data

Read in our Philadelphia mortgage data and take a peek at the first few rows.

* **Tip:** As always, census tract columns like to cause problems if they're read in as numbers. Make sure pandas reads it in as a string.

### Check your column types

I mentioned it above, but make sure `census_tract` is an object (a string) or merging isn't going to be any fun later on.

# Engineering and cleaning up features

## Income-related columns

> When we plotted the number of applicants, how much money they made and the size of the loan, we found that it skewed to the left, meaning the majority of applicants were clustered on the lower end of the income and loan amount scales. This was especially true for applicants of color. **We took the logarithm transformation of income and loan amount to normalize the distribution of those variables and limit the effect of extreme outliers.**

A few of the columns you'll need to calculate yourselves. **Calculate these values and assign them to three new columns.**

* Applicant’s adjusted annual income (includes co-applicant income)
* Adjusted loan amount
* Ratio between the loan amount and the applicant’s income

Instead of using the raw income and loan amount, you'll want the log of both income and loan amount. Call these new columns `log_income` and `log_loan_amount`. The third column will be `loan_income_ratio`.

* **Tip:** `np.log` gives you the logarithm

In [11]:
df['log_income'] = np.log(df.income)
df['log_loan_amount'] = np.log(df.loan_amount)
df['loan_income_ratio'] = np.log(df.loan_amount)/np.log(df.income)
df.head()


,census_tract,county_code,state_code,applicant_sex,income,loan_amount,loan_type,property_type,occupancy,action_type,loan_purpose,agency_code,tract_to_msa_income_percent,applicant_race,co_applicant_sex,log_income,log_loan_amount,loan_income_ratio
0,0101.00,101,42,3,26,5,1,1,1,4,2,5,97.09,6,5,3.258097,1.609438,0.493981
1,0264.00,101,42,2,26,40,1,1,1,4,2,5,98.27,3,5,3.258097,3.688879,1.132219
2,0281.00,101,42,2,22,20,1,1,1,5,2,5,72.28,6,5,3.091042,2.995732,0.969166
3,0158.00,101,42,2,57,36,1,1,1,5,3,5,105.87,6,5,4.043051,3.583519,0.886340
4,0358.00,101,42,1,80,34,1,1,1,1,3,5,139.62,5,2,4.382027,3.526361,0.804733


### Co-applicants

Right now we have a column about the co-applicant's sex (see the codebook for column details). We don't want the sex, though, we're interested in whether there is a co applicant or not. Use the co-applicant's sex to **create a new column called `co_applicant` that is either 'yes', 'no', or 'unknown'.**

* **Hint:** If the co-applicant's sex was not provided or is not applicable, count it as unknown.
* **Hint:** The easiest way is to use `.replace` on the co-applicant sex column, but store the result in your new column

Sex:
1 -- Male
2 -- Female
3 -- Information not provided by applicant in mail, Internet, or telephone application 4 -- Not applicable
5 -- No co-applicant

In [12]:
df['co_applicant'] = df.co_applicant_sex.replace([1,2,3,4,5], ['yes', 'yes', 'unknown', 'unknown','no' ])
df.head() 

,census_tract,county_code,state_code,applicant_sex,income,loan_amount,loan_type,property_type,occupancy,action_type,loan_purpose,agency_code,tract_to_msa_income_percent,applicant_race,co_applicant_sex,log_income,log_loan_amount,loan_income_ratio,co_applicant
0,0101.00,101,42,3,26,5,1,1,1,4,2,5,97.09,6,5,3.258097,1.609438,0.493981,no
1,0264.00,101,42,2,26,40,1,1,1,4,2,5,98.27,3,5,3.258097,3.688879,1.132219,no
2,0281.00,101,42,2,22,20,1,1,1,5,2,5,72.28,6,5,3.091042,2.995732,0.969166,no
3,0158.00,101,42,2,57,36,1,1,1,5,3,5,105.87,6,5,4.043051,3.583519,0.886340,no
4,0358.00,101,42,1,80,34,1,1,1,1,3,5,139.62,5,2,4.382027,3.526361,0.804733,yes


# Filter loan applicants

If you read the whitepaper - `lending_disparities_whitepaper_180214.pdf` - many filters are used to get to the target dataset for analysis.

> **Loan type**
>
> While we recognize the substantial presence of applicants of color in the FHA market, we focused on conventional home loans for several reasons.

> **Property type**
>
> Prospective borrowers submit loan applications for various types of structures: one- to four-unit properties, multifamily properties and manufactured homes. For this analysis, we focused on one- to four-unit properties.

> **Occupancy**
>
> We included only borrowers who said they planned to live in the house they were looking to buy. We did this to exclude developers or individuals who were buying property as an investment or to subsequently flip it.

> **Action Type**
>
> We wanted to look at the reasons lending institutions deny people a mortgage. After conversations with former officials at HUD, we decided to include only those applications that resulted in originations (action type 1) or denials (action type 3)

> **Income**
>
> An applicant’s income isn’t always reported in the data. In other cases, the data cuts off any incomes over \\$9.9 million and any loan amounts over \\$99.9 million, meaning there’s a value in the database, but it’s not precise. We focused only on those records where income and loan amount have an accurate estimation. This meant discarding about 1 percent of all conventional home loans in the country for 2016. [Note: I already edited this]
>
> When we plotted the number of applicants, how much money they made and the size of the loan, we found that it skewed to the left, meaning the majority of applicants were clustered on the lower end of the income and loan amount scales. This was especially true for applicants of color. We took the logarithm transformation of income and loan amount to normalize the distribution of those variables and limit the effect of extreme outliers.

> **Lien status**
>
> We included all cases in our analysis regardless of lien status.

> **Race and ethnicity**
>
> At first, we looked at race separate from ethnicity, but that approach introduced too many instances in which​ ​either the ethnicity or race was unknown. So we decided to combine race and ethnicity. Applicants who marked their ethnicity as Hispanic were grouped together as Hispanic/Latino regardless of race. Non-Hispanic applicants, as well as those who didn’t provide an ethnicity, were grouped together by race: non-Hispanic white, non-Hispanic black, etc. **[Note: This has already been taken care of]**

> **Loan purpose**
>
> We decided to look at home purchase, home improvement and refinance loans separately from each other. [Note: please look at **home purchase** loans.]

Use the text above (it's from the whitepaper) and the **2015HMDACodeSheet.pdf** code book to filter the dataset.

* **Tip:** there should be between 5-8 filters, depending on how you write them.

In [13]:
df=df[df['loan_type']==1]
df=df[df['property_type'] ==1]
df=df[df['loan_purpose'] == 1]
df=df[df['occupancy']==1]
df=df[df['action_type'].isin([1,3])]

When you're done filtering, save your dataframe as a "copy" with `df = df.copy()` (if it's called `df`, of course). This will prevent irritating warnings when you're trying to create new columns.

### Confirm that you have 10,107 loans with 19 columns

In [14]:
df.shape


(10107, 19)

### Create a "loan denied" column

Right now the `action_type` category reflects whether the loan was granted or not, and either has a value of `1` or `3`.

Create a new column called `loan_denied`, where the value is `0` if the loan was accepted and `1` if the loan was denied. **This will be our target for the machine learning algorithm.**

* **Tip:** You should have 8,878 successful loans and 1,229 denied loans

In [15]:
df["loan_denied"] = df["action_type"].replace({1: 0, 3: 1})
df.head()

,census_tract,county_code,state_code,applicant_sex,income,loan_amount,loan_type,property_type,occupancy,action_type,loan_purpose,agency_code,tract_to_msa_income_percent,applicant_race,co_applicant_sex,log_income,log_loan_amount,loan_income_ratio,co_applicant,loan_denied
42,4019.00,45,42,2,59,112,1,1,1,1,1,1,133.09,5,5,4.077537,4.718499,1.157193,no,0
43,4099.02,45,42,3,177,375,1,1,1,1,1,1,208.56,6,3,5.176150,5.926926,1.145045,unknown,0
46,4102.00,45,42,1,150,381,1,1,1,1,1,1,215.35,5,5,5.010635,5.942799,1.186037,no,0
48,0312.00,101,42,2,65,136,1,1,1,1,1,1,93.11,2,5,4.174387,4.912655,1.176857,no,0
51,4036.01,45,42,2,55,196,1,1,1,1,1,1,141.83,2,5,4.007333,5.278115,1.317114,no,0


In [16]:
df["loan_denied"].value_counts()

0    8878
1    1229
Name: loan_denied, dtype: int64

# Deal with categorical variables

Let's go ahead and take a look at our categorical variables:

* Applicant sex (male, female, na)
* Applicant race
* Mortgage agency
* Co-applicant (yes, no, unknown)

Before we do anything crazy, let's use the codebook to turn them into strings.

* **Tip:** We already did this with the `co_applicant` column, you only need to do the rest
* **Tip:** Just use `.replace`

Agency:
1 -- Office of the Comptroller of the Currency (OCC)
2 -- Federal Reserve System (FRS)
3 -- Federal Deposit Insurance Corporation (FDIC)
5 -- National Credit Union Administration (NCUA)
7 -- Department of Housing and Urban Development (HUD)
9 -- Consumer Financial Protection Bureau (CFPB)

Sex:
1 -- Male
2 -- Female
3 -- Information not provided by applicant in mail, Internet, or telephone application 4 -- Not applicable
5 -- No co-applicant

In [17]:
df['applicant_sex'] = df.applicant_sex.replace({1,2,3}, {'male', 'female','na'})
df['applicant_race'] = df.applicant_race.replace({1,2,3,4,5,6,99}, {'ai_an','asian','black','nh_opi','white','na','hispanic'})
df['agency_code'] = df.agency_code.replace({1,2,3,5,7,9}, {'OCC','FRS','FDIC','NCUA','HUD','CFPB'})
df.head()

,census_tract,county_code,state_code,applicant_sex,income,loan_amount,loan_type,property_type,occupancy,action_type,loan_purpose,agency_code,tract_to_msa_income_percent,applicant_race,co_applicant_sex,log_income,log_loan_amount,loan_income_ratio,co_applicant,loan_denied
42,4019.00,45,42,male,59,112,1,1,1,1,1,NCUA,133.09,ai_an,5,4.077537,4.718499,1.157193,no,0
43,4099.02,45,42,na,177,375,1,1,1,1,1,NCUA,208.56,hispanic,3,5.176150,5.926926,1.145045,unknown,0
46,4102.00,45,42,female,150,381,1,1,1,1,1,NCUA,215.35,ai_an,5,5.010635,5.942799,1.186037,no,0
48,0312.00,101,42,male,65,136,1,1,1,1,1,NCUA,93.11,white,5,4.174387,4.912655,1.176857,no,0
51,4036.01,45,42,male,55,196,1,1,1,1,1,NCUA,141.83,white,5,4.007333,5.278115,1.317114,no,0


In [18]:
df.shape


(10107, 20)

Double-check these columns match these values in the first three rows (and yes, you should have a lot of other columns, too).

|applicant_sex|agency_code|applicant_race|co_applicant|
|---|---|---|---|
|female|OCC|white|no|
|na|OCC|na|unknown|
|male|OCC|white|no|

## Dummy variables

Let's say we're at the end of the homework, and we have a column called `sex`, where `0` is female and `1` is male. After we've done our regression, we can look at the coefficient/odds ratio for `sex` and say something like **"being male gives you a 1.5x odds of being denied a loan."**

We can say this because we're looking at one column, and changing `sex` from `0` to `1` would turn the applicant male and give them a 1.5x chance of being denied (the odds ratio).

**But let's say we're looking at a column called `race` instead.** We could do the same `0`/`1` thing with white/minority, but what about white/black/asian? If we try to give them `0`/`1`/`2` our coefficient/odds ratio interpreation stops working, because we don't have a nice True/False dichotomy any more, it's now a *real number*.

* `0`: White
* `1`: Black
* `2`: Asian

Usually with numbers you can say "...for every increase of 1...", but we can't anymore - changing from White to Black (+1) isn't the same as changing from Black to Asian (+1). And you can't subtract Black from Asian to get White. And no, you also can't average together White and Asian to get Black. Just recognize that these aren't numbers, they're categories!

**How can we turn races off and on like we can turn the `sex` variable off and on?** A good option is to make *a `0`/`1` column for each race*. We can then flip each race off and on. These are called **dummy variables**.

In [19]:
pd.get_dummies(df.applicant_race, prefix='race').head()

,race_ai_an,race_asian,race_black,race_hispanic,race_na,race_nh_opi,race_white
42,1,0,0,0,0,0,0
43,0,0,0,1,0,0,0
46,1,0,0,0,0,0,0
48,0,0,0,0,0,0,1
51,0,0,0,0,0,0,1


Seems to take up a lot of space, but it works a lot better.

* The first person is white, so they have a `1` for white and a `0` for every other race
* The second person is N/A, so they have a `1` for N/A and a `0` for every other race
* The next three are white, asian, and asian, so they have a `1` under the appropriate column.

When you're looking at the regression output, each column has its own coefficient (and odds ratio). Since each race now has a column, **each race will also have its own odds ratio.** Asian would have one, Black would have one, Latino would have one - now we can look at the effect of each race separately. For example, you could then say something like "being Asian (e.g., `race_asian` going from `0` to `1`) gives you a 1.2x greater chance of being denied, and being Black gets you a 2.1x chance of being denied."

And no, you're never going to have more than one `1` in a row at the same time.

After you've created your dummy variables, there's one more step which has a real fun name: **one-hot encoding.**

### One-hot encoding

When we have two sexes - male and female - we can flip between them with one binary digit, `0` and `1`.

If we had three races - White, Asian, Black - using `pd.get_dummies` would make three columns, which makes sense on the surface. But why can we put TWO values in ONE column for sex, and it takes THREE columns for the THREE race values?

The truth is, it doesn't have to!

Instead of having three columns, we're only going to have two: **asian and black**. And if both of them are `0`? The applicant is white! This is called a **reference category**, and it means **the coefficients/odds ratios for asian and black are in reference to a white applicant.** So it isn't "being black gets you a 2.1x chance of being denied," it's *being black gets you a 2.1x chance of being denied compared to a white person*. For example:

|race_asian|race_black|person's race|
|---|---|---|
|1|0|Asian|
|0|1|Black|
|0|0|White|
|1|1|Not possible if your source is a single race column|

To create a one-hot encoded variable with a reference category, you write code like this:

In [20]:
pd.get_dummies(df.applicant_race, prefix='race').drop('race_white', axis=1).head()

,race_ai_an,race_asian,race_black,race_hispanic,race_na,race_nh_opi
42,1,0,0,0,0,0
43,0,0,0,1,0,0
46,1,0,0,0,0,0
48,0,0,0,0,0,0
51,0,0,0,0,0,0


> We usually use `.drop(columns=...)` to drop columns, but I'm using `axis=1` here because you should be familiar with it

### Make a one-hot encoded `sex` category with `female` as the reference category

You should end up with two columns: `sex_male` and `sex_na`.

In [21]:
pd.get_dummies(df.applicant_sex, prefix='sex').drop('sex_female', axis=1).head()

,sex_male,sex_na
42,1,0
43,0,1
46,0,0
48,1,0
51,1,0


## Using one-hot encoded columns

Since these one-hot encoded variables are standalone dataframes, we eventually need to combine them into our original dataframe.

We have four categorical variables - sex, race, co-applicant, and the loan agency - so we need you to **make four one-hot encoded variables**. Name them like this:

* `dummies_sex` - reference category of white
* `dummies_race` - reference category of female
* `dummies_co_applicant` - reference category of no
* `dummies_agency` - reference category of FDIC

Typically your reference category is the most common category, because it makes for the most interesting comparisons.

> **Tip:** if you're cutting and pasting from above, watch out for `.head()`
>
> **Tip:** After you've made them, use `.head(2)` to check the first couple rows of each to make sure they look okay

In [22]:
dummies_sex = pd.get_dummies(df.applicant_sex, prefix='sex').drop('sex_female', axis=1)
dummies_sex.head()

,sex_male,sex_na
42,1,0
43,0,1
46,0,0
48,1,0
51,1,0


In [23]:
dummies_race = pd.get_dummies(df.applicant_race, prefix='race').drop('race_white', axis=1)
dummies_race.head()

,race_ai_an,race_asian,race_black,race_hispanic,race_na,race_nh_opi
42,1,0,0,0,0,0
43,0,0,0,1,0,0
46,1,0,0,0,0,0
48,0,0,0,0,0,0
51,0,0,0,0,0,0


In [24]:
dummies_co_applicant = pd.get_dummies(df.co_applicant, prefix='co_applicant').drop('co_applicant_no', axis=1)

In [25]:
dummies_agency = pd.get_dummies(df.agency_code, prefix='agency').drop('agency_FDIC', axis=1)
dummies_agency.head(2)

,agency_CFPB,agency_FRS,agency_HUD,agency_NCUA,agency_OCC
42,0,0,0,1,0
43,0,0,0,1,0


## Cleaning up our old dataframe

Take a look at your original dataframe real quick.

In [26]:
df.head()

,census_tract,county_code,state_code,applicant_sex,income,loan_amount,loan_type,property_type,occupancy,action_type,loan_purpose,agency_code,tract_to_msa_income_percent,applicant_race,co_applicant_sex,log_income,log_loan_amount,loan_income_ratio,co_applicant,loan_denied
42,4019.00,45,42,male,59,112,1,1,1,1,1,NCUA,133.09,ai_an,5,4.077537,4.718499,1.157193,no,0
43,4099.02,45,42,na,177,375,1,1,1,1,1,NCUA,208.56,hispanic,3,5.176150,5.926926,1.145045,unknown,0
46,4102.00,45,42,female,150,381,1,1,1,1,1,NCUA,215.35,ai_an,5,5.010635,5.942799,1.186037,no,0
48,0312.00,101,42,male,65,136,1,1,1,1,1,NCUA,93.11,white,5,4.174387,4.912655,1.176857,no,0
51,4036.01,45,42,male,55,196,1,1,1,1,1,NCUA,141.83,white,5,4.007333,5.278115,1.317114,no,0


In [27]:
df.shape


(10107, 20)

In [28]:
df.columns

Index(['census_tract', 'county_code', 'state_code', 'applicant_sex', 'income',
       'loan_amount', 'loan_type', 'property_type', 'occupancy', 'action_type',
       'loan_purpose', 'agency_code', 'tract_to_msa_income_percent',
       'applicant_race', 'co_applicant_sex', 'log_income', 'log_loan_amount',
       'loan_income_ratio', 'co_applicant', 'loan_denied'],
      dtype='object')

In [29]:
df['loan_income_ratio']=df['loan_amount']/df['income']

We don't need all of those columns! If we look at the list of columns we'll be using for the regression:

* Race/Ethnicity
* Sex
* Whether there was a co-applicant
* Applicant’s annual income (includes co-applicant income)
* Loan amount
* Ratio between the loan amount and the applicant’s income
* Ratio between the median income of the census tract and the median income of the metro area
* Racial and ethnic breakdown by percentage for each census tract
* Regulating agency of the lending institution

We can keep anything in that list, and remove everything else. For example, we can drop the variables we used to create the dummy variables, as we'll be adding the one-hot encoded versions in for the next step.

For "Racial and ethnic breakdown by percentage for each census tract" we'll need to join with some census data later, so we need to also keep census tract, county code and state code.

**Build a new dataframe with only the columns we're interested in, call it `numeric`.** We're calling it `numeric` because it's mostly numeric columns after the categorical ones have been removed.

> **Tip:** You can either use `.drop(columns=` to remove unwanted columns or `df = df[['col1', 'col2', ... 'col12']]` to only select the ones you're interseted in

In [30]:
numeric = df.drop(columns=['loan_type','property_type','occupancy','loan_purpose',
                 'loan_amount','income', 'applicant_sex','applicant_race',
                'co_applicant_sex','co_applicant', 'agency_code', 'action_type'])
                



Confirm that `numeric` has 8 columns.

In [31]:
numeric.head()

,census_tract,county_code,state_code,tract_to_msa_income_percent,log_income,log_loan_amount,loan_income_ratio,loan_denied
42,4019.00,45,42,133.09,4.077537,4.718499,1.898305,0
43,4099.02,45,42,208.56,5.176150,5.926926,2.118644,0
46,4102.00,45,42,215.35,5.010635,5.942799,2.540000,0
48,0312.00,101,42,93.11,4.174387,4.912655,2.092308,0
51,4036.01,45,42,141.83,4.007333,5.278115,3.563636,0


In [32]:
numeric.shape

(10107, 8)

### Combining our features

We now have 1 dataframe of numeric features (and some merge columns), and 4 one-hot-encoded variables (each with their own dataframe). Combine all five dataframes into one large dataframe called `loan_features`.

In [33]:
loan_features = pd.concat([numeric, dummies_sex, dummies_race, dummies_co_applicant, dummies_agency], axis=1)
loan_features.head()


,census_tract,county_code,state_code,tract_to_msa_income_percent,log_income,log_loan_amount,loan_income_ratio,loan_denied,sex_male,sex_na,race_ai_an,race_asian,race_black,race_hispanic,race_na,race_nh_opi,co_applicant_unknown,co_applicant_yes,agency_CFPB,agency_FRS,agency_HUD,agency_NCUA,agency_OCC
42,4019.00,45,42,133.09,4.077537,4.718499,1.898305,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0
43,4099.02,45,42,208.56,5.176150,5.926926,2.118644,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0
46,4102.00,45,42,215.35,5.010635,5.942799,2.540000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
48,0312.00,101,42,93.11,4.174387,4.912655,2.092308,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
51,4036.01,45,42,141.83,4.007333,5.278115,3.563636,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [34]:
loan_features_bis = pd.concat([numeric, dummies_sex, dummies_race, dummies_co_applicant, dummies_agency]).groupby(level=0).any().astype(int)
loan_features.head()

/Users/boostigooff/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,census_tract,county_code,state_code,tract_to_msa_income_percent,log_income,log_loan_amount,loan_income_ratio,loan_denied,sex_male,sex_na,race_ai_an,race_asian,race_black,race_hispanic,race_na,race_nh_opi,co_applicant_unknown,co_applicant_yes,agency_CFPB,agency_FRS,agency_HUD,agency_NCUA,agency_OCC
42,4019.00,45,42,133.09,4.077537,4.718499,1.898305,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0
43,4099.02,45,42,208.56,5.176150,5.926926,2.118644,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0
46,4102.00,45,42,215.35,5.010635,5.942799,2.540000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
48,0312.00,101,42,93.11,4.174387,4.912655,2.092308,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
51,4036.01,45,42,141.83,4.007333,5.278115,3.563636,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0


Confirm that `loan_features` has 10,107 rows and 23 columns.

In [35]:
loan_features.shape

(10107, 23)

In [36]:
loan_features_bis.shape

(10107, 23)

# Census data

Now we just need the final piece to the puzzle, the census data. Read in the census data file, calling the dataframe `census`.

> **Tip:** As always, be sure to read the tract column in as a string. Interestingly, this time we _don't_ need to worry about the state or county codes in the same way.
>
> **Tip:** You're going to encounter a problem that you find every time you read in a file from the US government!

In [37]:
census=pd.read_csv('data/nhgis0007_ds215_20155_2015_tract.csv',encoding='latin-1',dtype={'TRACTA':'str'})
census.head()

,GISJOIN,YEAR,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,PLACEA,TRACTA,BLKGRPA,CONCITA,AIANHHA,RES_ONLYA,TRUSTA,AITSCEA,ANRCA,CBSAA,CSAA,METDIVA,NECTAA,CNECTAA,NECTADIVA,UAA,CDCURRA,SLDUA,SLDLA,ZCTA5A,SUBMCDA,SDELMA,SDSECA,SDUNIA,PUMA5A,BTTRA,BTBGA,NAME_E,ADK5E001,ADK5E002,ADK5E003,ADK5E004,ADK5E005,ADK5E006,ADK5E007,ADK5E008,ADK5E009,ADK5E010,ADK5E011,ADK5E012,ADK5E013,ADK5E014,ADK5E015,ADK5E016,ADK5E017,ADK5E018,ADK5E019,ADK5E020,ADK5E021,NAME_M,ADK5M001,ADK5M002,ADK5M003,ADK5M004,ADK5M005,ADK5M006,ADK5M007,ADK5M008,ADK5M009,ADK5M010,ADK5M011,ADK5M012,ADK5M013,ADK5M014,ADK5M015,ADK5M016,ADK5M017,ADK5M018,ADK5M019,ADK5M020,ADK5M021
0,G0100010020100,2011-2015,NaN,NaN,Alabama,1,Autauga County,1,NaN,NaN,020100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Census Tract 201, Autauga County, Alabama",1948,1931,1703,150,6,12,0,0,60,0,60,17,17,0,0,0,0,0,0,0,0,"Census Tract 201, Autauga County, Alabama",203,212,229,126,8,16,11,11,44,11,44,21,21,11,11,11,11,11,11,11,11
1,G0100010020200,2011-2015,NaN,NaN,Alabama,1,Autauga County,1,NaN,NaN,020200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Census Tract 202, Autauga County, Alabama",2156,2139,872,1149,0,50,0,0,68,0,68,17,14,0,0,0,0,3,0,0,0,"Census Tract 202, Autauga County, Alabama",268,268,207,250,11,61,11,11,62,11,62,25,23,11,11,11,11,7,11,11,11
2,G0100010020300,2011-2015,NaN,NaN,Alabama,1,Autauga County,1,NaN,NaN,020300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Census Tract 203, Autauga County, Alabama",2968,2968,2212,551,15,41,8,0,141,0,141,0,0,0,0,0,0,0,0,0,0,"Census Tract 203, Autauga County, Alabama",404,404,372,190,22,62,14,11,135,11,135,11,11,11,11,11,11,11,11,11,11
3,G0100010020400,2011-2015,NaN,NaN,Alabama,1,Autauga County,1,NaN,NaN,020400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Census Tract 204, Autauga County, Alabama",4423,3959,3662,162,69,0,0,48,18,5,13,464,30,42,0,0,0,372,20,20,0,"Census Tract 204, Autauga County, Alabama",493,353,343,133,81,11,11,82,17,8,16,437,33,50,11,11,11,456,29,29,11
4,G0100010020500,2011-2015,NaN,NaN,Alabama,1,Autauga County,1,NaN,NaN,020500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Census Tract 205, Autauga County, Alabama",10763,10683,7368,2674,0,412,0,0,229,49,180,80,80,0,0,0,0,0,0,0,0,"Census Tract 205, Autauga County, Alabama",624,616,796,742,18,242,18,18,166,74,149,71,71,18,18,18,18,18,18,18,18


## Rename some columns

If you like to keep your data extra clean, feel free to rename the columns you're interested in. If not, feel free to skip it!

> **Tip:** Make sure you're using the estimates columns, not the margin of error columns

## Computed columns

According to Reveal's regression output, you'll want to create the following columns:

* Percent Black in tract
* Percent Hispanic/Latino in tract (I hope you know how Hispanic/Latino + census data works by now)
* Percent Asian in tract
* Percent Native American in tract
* Percent Native Hawaiian in tract

Notice that we don't include percent white - **because all of the other columns add up to percent white, we ignore it!** It's similar to a reference category.

> If we want to use buzzwords here, the technical reason we're not using percent white is called **collinearity.** We'll talk more about it on Friday.

In [38]:
census=census.rename(columns={'ADK5E001':'Total','ADK5E004':'Black',
'ADK5E012':'Hispanic/Latino',
'ADK5E006':'Asian',
'ADK5E005':'American Indian',
'ADK5E007':'Native Hawaiian'})

In [39]:
census.dtypes

GISJOIN       object
YEAR          object
REGIONA      float64
DIVISIONA    float64
STATE         object
STATEA         int64
COUNTY        object
COUNTYA        int64
COUSUBA      float64
PLACEA       float64
TRACTA        object
BLKGRPA      float64
CONCITA      float64
AIANHHA      float64
RES_ONLYA    float64
TRUSTA       float64
AITSCEA      float64
ANRCA        float64
CBSAA        float64
CSAA         float64
METDIVA      float64
NECTAA       float64
CNECTAA      float64
NECTADIVA    float64
UAA          float64
CDCURRA      float64
SLDUA        float64
SLDLA        float64
ZCTA5A       float64
SUBMCDA      float64
              ...   
ADK5E014       int64
ADK5E015       int64
ADK5E016       int64
ADK5E017       int64
ADK5E018       int64
ADK5E019       int64
ADK5E020       int64
ADK5E021       int64
NAME_M        object
ADK5M001       int64
ADK5M002       int64
ADK5M003       int64
ADK5M004       int64
ADK5M005       int64
ADK5M006       int64
ADK5M007       int64
ADK5M008     

In [40]:
census['pct_black']=census['Black']/census['Total']*100
census['pct_hispanic']=census['Hispanic/Latino']/census['Total']*100
census['pct_asian']=census['Asian']/census['Total']*100
census['pct_amer_indian']=census['American Indian']/census['Total']*100
census['pct_pac_islander']=census['Native Hawaiian']/census['Total']*100

## Only keep what we need to join and process

We're only interested in the percentage columns that we computed. Create a new dataframe called `census_features` that is only those columns along with the one we'll need for joining with the mortgage data.

> * **Tip:** Remember we saved state, county and tract codes when working on the loan data

In [41]:
census_features=census[['STATEA','COUNTYA','TRACTA','pct_black','pct_hispanic','pct_asian','pct_amer_indian','pct_pac_islander']]

In [42]:
census_features.head()

,STATEA,COUNTYA,TRACTA,pct_black,pct_hispanic,pct_asian,pct_amer_indian,pct_pac_islander
0,1,1,020100,7.700205,0.872690,0.616016,0.308008,0.000000
1,1,1,020200,53.293135,0.788497,2.319109,0.000000,0.000000
2,1,1,020300,18.564690,0.000000,1.381402,0.505391,0.269542
3,1,1,020400,3.662672,10.490617,0.000000,1.560027,0.000000
4,1,1,020500,24.844374,0.743287,3.827929,0.000000,0.000000


Confirm that your first few rows look something like this:
    
|STATEA|COUNTYA|TRACTA|pct_hispanic|pct_black|pct_amer_indian|pct_asian|pct_pac_islander|
|---|---|---|---|---|---|---|---|
|1|1|020100|0.872690|7.700205|0.308008|0.616016|0.000000|
|1|1|020200|0.788497|53.293135|0.000000|2.319109|0.000000|
|1|1|020300|0.000000|18.564690|0.505391|1.381402|0.269542|
|1|1|020400|10.490617|3.662672|1.560027|0.000000|0.000000|
|1|1|020500|0.743287|24.844374|0.000000|3.827929|0.000000|

Your column headers might be different but your numbers should match.

# Merge datasets

Merge `loan_features` and `census_features` into a new dataframe called `merged`.

Unfortunately something is a little different between our `loan_features` and `census_features` census tract columns. You'll need to fix it before you can merge.

## Cleaning

In [43]:
loan_features.census_tract = loan_features.census_tract.astype(str)

In [44]:
loan_features.dtypes

census_tract                    object
county_code                      int64
state_code                       int64
tract_to_msa_income_percent    float64
log_income                     float64
log_loan_amount                float64
loan_income_ratio              float64
loan_denied                      int64
sex_male                         uint8
sex_na                           uint8
race_ai_an                       uint8
race_asian                       uint8
race_black                       uint8
race_hispanic                    uint8
race_na                          uint8
race_nh_opi                      uint8
co_applicant_unknown             uint8
co_applicant_yes                 uint8
agency_CFPB                      uint8
agency_FRS                       uint8
agency_HUD                       uint8
agency_NCUA                      uint8
agency_OCC                       uint8
dtype: object

In [45]:
loan_features.census_tract=loan_features.census_tract.str.replace('.','')

In [46]:
df.census_tract.head()

42    4019.00
43    4099.02
46    4102.00
48    0312.00
51    4036.01
Name: census_tract, dtype: object

## Do the merge

In [47]:
merged = loan_features.merge(census_features,right_on=['STATEA','COUNTYA','TRACTA'],left_on=['state_code','county_code','census_tract'])

merged

,census_tract,county_code,state_code,tract_to_msa_income_percent,log_income,log_loan_amount,loan_income_ratio,loan_denied,sex_male,sex_na,race_ai_an,race_asian,race_black,race_hispanic,race_na,race_nh_opi,co_applicant_unknown,co_applicant_yes,agency_CFPB,agency_FRS,agency_HUD,agency_NCUA,agency_OCC,STATEA,COUNTYA,TRACTA,pct_black,pct_hispanic,pct_asian,pct_amer_indian,pct_pac_islander
0,401900,45,42,133.09,4.077537,4.718499,1.898305,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,42,45,401900,40.744252,0.900687,5.309315,0.000000,0.000000
1,401900,45,42,133.09,4.143135,5.257495,3.047619,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,42,45,401900,40.744252,0.900687,5.309315,0.000000,0.000000
2,401900,45,42,133.09,4.382027,4.653960,1.312500,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,42,45,401900,40.744252,0.900687,5.309315,0.000000,0.000000
3,401900,45,42,133.09,4.430817,4.852030,1.523810,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,42,45,401900,40.744252,0.900687,5.309315,0.000000,0.000000
4,401900,45,42,133.09,3.258097,5.123964,6.461538,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,42,45,401900,40.744252,0.900687,5.309315,0.000000,0.000000
5,401900,45,42,133.09,3.465736,4.521789,2.875000,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,42,45,401900,40.744252,0.900687,5.309315,0.000000,0.000000
6,401900,45,42,133.09,4.736198,5.247024,1.666667,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,42,45,401900,40.744252,0.900687,5.309315,0.000000,0.000000
7,401900,45,42,133.09,3.850148,5.220356,3.936170,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,42,45,401900,40.744252,0.900687,5.309315,0.000000,0.000000
8,401900,45,42,133.09,4.043051,5.560682,4.561404,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,42,45,401900,40.744252,0.900687,5.309315,0.000000,0.000000
9,401900,45,42,133.09,4.094345,4.867534,2.166667,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,42,45,401900,40.744252,0.900687,5.309315,0.000000,0.000000


Confirm you have 10107 rows and 31 columns in the merged dataframe.

In [48]:
merged.shape

(10107, 31)

# Our final dataframe

Drop all of the columns we merged on and save it as `train_df`.

In [49]:

train_df = merged.drop(columns=['STATEA','COUNTYA','TRACTA','state_code','county_code','census_tract'])


Confirm that `train_df` has 10107 rows and 25 columns.

In [50]:
train_df.shape

(10107, 25)

In [51]:
train_df.head()

,tract_to_msa_income_percent,log_income,log_loan_amount,loan_income_ratio,loan_denied,sex_male,sex_na,race_ai_an,race_asian,race_black,race_hispanic,race_na,race_nh_opi,co_applicant_unknown,co_applicant_yes,agency_CFPB,agency_FRS,agency_HUD,agency_NCUA,agency_OCC,pct_black,pct_hispanic,pct_asian,pct_amer_indian,pct_pac_islander
0,133.09,4.077537,4.718499,1.898305,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,40.744252,0.900687,5.309315,0.0,0.0
1,133.09,4.143135,5.257495,3.047619,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,40.744252,0.900687,5.309315,0.0,0.0
2,133.09,4.382027,4.653960,1.312500,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,40.744252,0.900687,5.309315,0.0,0.0
3,133.09,4.430817,4.852030,1.523810,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,40.744252,0.900687,5.309315,0.0,0.0
4,133.09,3.258097,5.123964,6.461538,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,40.744252,0.900687,5.309315,0.0,0.0


## Final cleanup

Because we can't have missing data before we run a regression, check the size of `train_df`, then drop any missing data and check the size again. **Confirm you don't lose any rows.**

In [52]:
train_df = train_df.dropna()


In [53]:
train_df.shape

(10107, 25)

In [54]:
 train_df.loan_denied.value_counts(dropna=False) 

0    8878
1    1229
Name: loan_denied, dtype: int64

# Performing our regression

## Try with statsmodels

First try to run a linear regression with statsmodels, because even though sometimes it complains and breaks, the output just looks *so nice*. Instead of `sm.OLS` we'll use `sm.Logit`.

In [55]:
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [59]:
X = train_df.drop(columns='loan_denied')
y = train_df.loan_denied

mod = sm.Logit(y, X)
res = mod.fit()
res.summary() 

Optimization terminated successfully.
         Current function value: 0.334036
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            loan_denied   No. Observations:                10107
Model:                          Logit   Df Residuals:                    10083
Method:                           MLE   Df Model:                           23
Date:                Tue, 23 Jul 2019   Pseudo R-squ.:                 0.09744
Time:                        18:40:32   Log-Likelihood:                -3376.1
converged:                       True   LL-Null:                       -3740.6
Covariance Type:            nonrobust   LLR p-value:                3.479e-139
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
tract_to_msa_income_percent     0.0017      0.001      2.566      0.010       0.000       0.003
log_income                     -0.3695      0.066     -5.612      0.000      -0.499      -0.240
log_loan_amount                -0.2400      0.053     -4.548      0.000      -0.343      -0.137
loan_income_ratio               0.0102      0.007      1.425      0.154      -0.004       0.024
sex_male                       -0.1267      0.070     -1.811      0.070      -0.264       0.010
sex_na                         -0.2354      0.174     -1.353      0.176      -0.576       0.106
race_ai_an                     -0.3907      0.103     -3.789      0.000      -0.593      -0.189
race_asian                      0.6634      0.596      1.114      0.265      -0.504       1.831
race_black                      0.3550      0.135      2.624      0.009       0.090       0.620
race_hispanic                   0.0656      0.145      0.453      0.651      -0.218       0.349
race_na                        -0.0768      0.176     -0.437      0.662      -0.421       0.268
race_nh_opi                     0.7043      0.469      1.501      0.133      -0.215       1.624
co_applicant_unknown            0.4079      0.206      1.980      0.048       0.004       0.812
co_applicant_yes               -0.0897      0.078     -1.153      0.249      -0.242       0.063
agency_CFPB                     1.3511      0.171      7.899      0.000       1.016       1.686
agency_FRS                      1.1631      0.163      7.130      0.000       0.843       1.483
agency_HUD                      0.0485      0.193      0.251      0.802      -0.330       0.427
agency_NCUA                     0.3661      0.211      1.738      0.082      -0.047       0.779
agency_OCC                      0.1703      0.164      1.042      0.297      -0.150       0.491
pct_black                       0.0059      0.002      3.901      0.000       0.003       0.009
pct_hispanic                    0.0065      0.003      1.947      0.052   -4.41e-05       0.013
pct_asian                       0.0101      0.004      2.345      0.019       0.002       0.019
pct_amer_indian                -0.2520      0.097     -2.609      0.009      -0.441      -0.063
pct_pac_islander                0.0929      0.159      0.584      0.559      -0.219       0.405
===============================================================================================
"""

## Try again with sci-kit learn

But I'll be honest, I like sklearn a *lot lot lot* better. Using the coefficient to build a dataframe just seems so *nice*.

> **Tip:** When you build your model, use `LogisticRegression(C=1e9, solver='lbfgs', max_iter=4000)` - for if you don't increase `max_iter` (how long/hard it works) it'll complain it can't find an answer.

In [57]:
X = train_df.drop(columns='loan_denied')
y = train_df.loan_denied
clf = LogisticRegression(C=1e9, solver='lbfgs', max_iter=4000)
clf.fit(X, y)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=4000, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### Getting your coefficients and odds ratios

After you run your regression **using sklearn**, you can use code like the below to print out an ordered list of features, coefficients, and odds ratios.

```python
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients)
}).sort_values(by='odds ratio', ascending=False)
```

In [60]:
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient (log odds ratio),odds ratio
14,agency_CFPB,1.445128,4.242396
15,agency_FRS,1.244462,3.471066
11,race_nh_opi,0.675701,1.965411
7,race_asian,0.493780,1.638498
17,agency_NCUA,0.447940,1.565085
12,co_applicant_unknown,0.394333,1.483395
8,race_black,0.360961,1.434707
18,agency_OCC,0.256651,1.292594
16,agency_HUD,0.137295,1.147167
23,pct_pac_islander,0.090738,1.094982


### Wait, what's the odds ratio again?

It's how much that variable affects the outcome **if all other variables stay the same.**

# Interpreting and thinking about the analysis

### Question 1

Our results aren't exactly the same as Reveal's, as I pulled a slightly different number of rows from the database and I'm not sure what exact dataset they used for census information. How are we feeling about this reproduction? **You might want check their 2015 results in the whitepaper.**

### Question 2

In the opening paragraph to the flagship piece, [Aaron and Emmanuel write](https://www.revealnews.org/article/for-people-of-color-banks-are-shutting-the-door-to-homeownership/):

> Fifty years after the federal Fair Housing Act banned racial discrimination in lending, African Americans and Latinos continue to be routinely denied conventional mortgage loans at rates far higher than their white counterparts.

If you look at the results, Hawaiians/Pacific Islanders (and maybe Native Americans) have an even higher odds ratio. **Why do they choose to talk about African Americans and Latinos instead?**

### Question 3

Write a sentence expressing the meaning of the **odds ratio** statistic for Black mortgage applicants. Find a line in [the Reveal piece](https://www.revealnews.org/article/for-people-of-color-banks-are-shutting-the-door-to-homeownership/) where they use the odds ratio.

### Question 4

Write a similar sentence about men.

### Question 5

Why did Aaron and Emmanuel choose to include the loan-to-income ratio statistic? **You might want to read the whitepaper.**

### Question 6

Credit score is a common reason why loans are denied. Why are credit scores not included in our analysis? **You might want to read the whitepaper.**

### Question 7

This data was just sitting out there for anyone to look at, they didn't even need to FOIA it. Why do you think this issue had not come up before Reveal's analysis?

### Question 8

As a result of this series, [a lot has happened](https://www.revealnews.org/blog/we-exposed-modern-day-redlining-in-61-cities-find-out-whats-happened-since/), although [recent changes don't look so good](https://www.revealnews.org/blog/cfpb-moves-to-limit-home-loan-data/). If you were reporting this story, what groups of people would you want to talk to in order to make sure you're getting the story right?

### Question 9

When they were consulting experts, Aaron and Emmanuel received a lot of conflicting accounts about whether they should include the "N/A" values for race (they ended up including it). If the experts disagreed about something like that, why do you think they went forward with their analysis?

### Question 10

What if we were working on this story, and our logistic regression or input dataset were flawed? What would be the repercussions?